In [26]:
%matplotlib ipympl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, welch
import sys
import pywt
import util

# interactive plotting with %matplotlib notebook

!ls -lh ../recordings

total 52968
-rw-r--r--@ 1 nickgeoca  staff   2.2M Mar 21 21:40 2025-03-21_16-10_gain24_boardAds1299_vref4.5.csv
-rw-r--r--@ 1 nickgeoca  staff    12M Apr  1 15:42 8ch.csv
-rw-r--r--@ 1 nickgeoca  staff    10M Apr  1 20:23 basic.ipynb
-rw-r--r--@ 1 nickgeoca  staff   754K Mar 27 16:23 battery.csv
-rw-r--r--@ 1 nickgeoca  staff   494K Mar 25 18:53 march-25-test.csv


In [27]:
print('Load data and get properties\n')

fname = 'battery.csv'
df = pd.read_csv(f"../recordings/{fname}")
fname = '2025-03-21_16-10_gain24_boardAds1299_vref4.5.csv'

board, gain, Fs, vref, avss, resolution, vmid = util.get_board_attributes(fname)
channel_count = len([i for i in list(df.columns) if i.find('_raw_sample') !=-1])
len_minutes = (df['timestamp'].iloc[-1]-df['timestamp'].iloc[0])/1000000. / 60.

print(f"vref={vref}V, avss={avss}V, gain={gain}, sample_rate={Fs}hz, vmid={vmid}, resolution={resolution} bit")
print(f"\nchannel count= {channel_count}")
print(f"\ndata length= {int(len_minutes)}min {int((len_minutes%1)*60)}sec")

Load data and get properties

vref=4.5V, avss=0V, gain=24.0, sample_rate=250.0hz, vmid=2.25, resolution=24 bit

channel count= 4

data length= 0min 39sec


In [28]:
df = util.add_time_and_raw_voltage_columns(df, fname)

# Trime first and last
remove_first_seconds = 0
remove_last_seconds = 0
max_time = df['time_sec'].max()
df = df[(df['time_sec'] >= remove_first_seconds) & (df['time_sec'] <= (max_time - remove_last_seconds))].reset_index(drop=True)

df

added these new columns, ['time_sec', 'ch1_raw_voltage', 'ch2_raw_voltage', 'ch3_raw_voltage', 'ch4_raw_voltage']


,timestamp,ch1_voltage,ch2_voltage,ch3_voltage,ch4_voltage,ch1_raw_sample,ch2_raw_sample,ch3_raw_sample,ch4_raw_sample,time_sec,ch1_raw_voltage,ch2_raw_voltage,ch3_raw_voltage,ch4_raw_voltage
0,1743114112565138,5.010070e-07,-1.552363e-05,0.0,0.0,-7958227,1643025,0,0,0.000,2.072120,2.286724,2.25,2.25
1,1743114112569138,1.811448e-06,8.174743e-07,0.0,0.0,-7961838,1600167,0,0,0.004,2.072039,2.285767,2.25,2.25
2,1743114112573138,4.460768e-07,-1.365484e-07,0.0,0.0,-7959716,1603078,0,0,0.008,2.072086,2.285832,2.25,2.25
3,1743114112577138,-1.317618e-06,-1.541293e-05,0.0,0.0,-7955941,1630208,0,0,0.012,2.072171,2.286438,2.25,2.25
4,1743114112581138,-8.137809e-07,-1.825686e-05,0.0,0.0,-7957461,1643923,0,0,0.016,2.072137,2.286745,2.25,2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9915,1743114152225138,8.136446e-06,9.020668e-05,0.0,0.0,-7956767,1591561,0,0,39.660,2.072152,2.285574,2.25,2.25
9916,1743114152229138,7.415625e-06,8.330658e-05,0.0,0.0,-7962849,1554090,0,0,39.664,2.072016,2.284737,2.25,2.25
9917,1743114152233138,4.114983e-06,5.957759e-05,0.0,0.0,-7961019,1542451,0,0,39.668,2.072057,2.284476,2.25,2.25
9918,1743114152237138,-2.843178e-07,1.470723e-05,0.0,0.0,-7957107,1564946,0,0,39.672,2.072145,2.284979,2.25,2.25


In [29]:
eeg_single_channel = df['ch2_raw_voltage'].values
eeg_single_channel

array([2.28672447, 2.28576652, 2.28583159, ..., 2.28447647, 2.28497927,
       2.28588194])

In [30]:
      
import torch
import librosa # For resampling (pip install librosa)
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2Model
# Or use HubertModel, AutoModel etc.

# --- 1. Load Pre-trained Model & Processor ---
# Using a base model for speed/manageability
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name)
model.eval() # Set to evaluation mode

# Get the expected sampling rate
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"Model expects input sampling rate: {target_sampling_rate} Hz")

# --- 2. Load and Preprocess Your EEG Data ---
# !! CRITICAL STEP: Adapt this to your actual data loading (e.g., using MNE) !!
# Example: Load a dummy EEG segment (replace with your real data)
original_sampling_rate = 250 # Your EEG data's sampling rate (e.g., 256 Hz)
n_channels = 1 # Example number of EEG channels
n_samples = original_sampling_rate * 10 # 10 seconds of data

# Dummy data: Replace with loading from FIF file, selecting channels etc.
# dummy_eeg_data = df[].values

# --- 3. **Adaptation/Hack for Quick Test** ---
#   a) Select ONE channel (simplest case, or average/process independently later)
# eeg_single_channel = dummy_eeg_data[0, :] # Taking the first channel

#   b) Resample to the model's expected rate (!! LOSES EEG INFO !!)
#      This is a HACK for compatibility, not ideal for real analysis.
print(f"Resampling EEG from {original_sampling_rate} Hz to {target_sampling_rate} Hz...")
eeg_resampled = librosa.resample(eeg_single_channel,
                                 orig_sr=original_sampling_rate,
                                 target_sr=target_sampling_rate)
print(f"Resampled EEG shape: {eeg_resampled.shape}")

# --- 4. Process and Extract Features ---
# Use the processor to prepare the input format
# The processor handles normalization and padding/truncation if needed
input_values = processor(eeg_resampled,
                         sampling_rate=target_sampling_rate,
                         return_tensors="pt").input_values # Get PyTorch tensors

print(f"Shape of model input: {input_values.shape}") # (batch=1, num_samples)

# --- 5. Run Inference ---
with torch.no_grad():
    outputs = model(input_values)
    # Extract the features from the last hidden state
    # Shape: (batch_size, sequence_length, hidden_size)
    # sequence_length is downsampled by the CNN layers
    features = outputs.last_hidden_state

print(f"Shape of extracted features: {features.shape}")

# --- 6. Use the Features ---
# 'features' now contains the extracted representation from Wav2Vec2
# You can now:
# - Average across the sequence length for a single vector per segment: features.mean(dim=1)
# - Feed the sequence into an LSTM/Transformer/Classifier for downstream tasks
# - Visualize using dimensionality reduction (PCA, UMAP on features.squeeze().numpy())

segment_feature_vector = features.mean(dim=1)
print(f"Shape of segment-level feature vector: {segment_feature_vector.shape}") # (1, 768 for base model)

    

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model expects input sampling rate: 16000 Hz
Resampling EEG from 250 Hz to 16000 Hz...
Resampled EEG shape: (634880,)
Shape of model input: torch.Size([1, 634880])
Shape of extracted features: torch.Size([1, 1983, 768])
Shape of segment-level feature vector: torch.Size([1, 768])


In [31]:
segment_feature_vector

tensor([[-1.1107e-01,  5.4887e-02, -1.3773e-02,  2.9985e-04, -1.5900e-01,
         -8.6085e-02,  4.1208e-02, -1.0203e-02, -2.0229e-01, -3.3465e-01,
          9.6472e-03, -5.5998e-02,  3.6114e-02,  4.7132e-02, -2.0519e-01,
         -2.1843e-02, -1.9791e-01,  3.8042e-01, -1.4231e-02,  4.0069e-03,
         -1.5623e-01,  1.5374e-01,  1.4869e-01,  6.5259e-03,  4.1757e-01,
         -1.1493e-01, -3.4415e-01,  8.7998e-02,  2.0316e-02, -2.9996e-02,
          1.3455e-01, -1.0147e-02, -1.4127e-02, -5.8496e-02, -1.7997e-01,
         -1.3405e-01,  1.4362e-01, -4.1667e-01, -4.5271e-02,  3.9472e-02,
         -1.5835e-01, -6.3869e-03, -2.2225e-01,  2.9312e-02, -6.2875e-02,
          6.6688e-02, -7.4469e-02,  8.0738e-02,  2.8213e-02,  9.1497e-03,
         -1.6970e-01, -1.3624e-01,  2.8621e-01, -6.7713e-02,  2.8663e-02,
          1.5700e-02, -1.0592e-02, -5.5392e-01, -1.9328e-01, -9.8990e-02,
          5.0199e-02, -1.4369e-01,  2.2173e-02,  5.6845e-03, -5.5345e-02,
          4.0862e-02, -6.0448e-03, -2.